In [2]:
# Testing Paru

import cv2
import numpy as np
import requests

def prepare_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224,224))
    img = img.astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=0)
    return img.tolist()

image_path = './img_test.png'
image = prepare_image(image_path)

data = {"instances": image}  # Konversi ke daftar Python agar dapat di-serialize menjadi JSON

url = 'https://medical-math-models.onrender.com/v1/models/model_paru:predict'

response = requests.post(url, json=data)

predict = response.json()
class_labels = np.array(['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema',
       'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration',
       'Mass', 'Nodule', 'Normal', 'Pleural_Thickening', 'Pneumonia',
       'Pneumothorax'])
predicted_label = class_labels[np.argmax(predict['predictions'])]

predicted_label

'Hernia'